<a href="https://colab.research.google.com/github/nitiwat-tdg/knowledge_sharing/blob/main/ks01_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial modified form https://www.tensorflow.org/tutorials/images/cnn

In [ ]:
! nvidia-smi

In [ ]:
import math
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from typing import Iterator, Tuple
from tensorflow.raw_ops import Softmax
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


# set seed
seed = 123
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# Define class names

CLASS_NAMES = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
  ]

In [ ]:
# Load data

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [ ]:
# Check dataset shape

print("train_images.shape:", train_images.shape)
print("test_images.shape:", test_images.shape)

print("train_labels.shape:", train_labels.shape)
print("test_labels.shape:", test_labels.shape)

In [ ]:
# Visualize label

print("train_labels:", train_labels)

In [ ]:
# Visualize Data

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(CLASS_NAMES[train_labels[i][0]])
    
plt.show()

In [ ]:
# Create Model

model = models.Sequential()

# Conv Layers + Pooling Layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Fully Connected Layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# Visualize Model

model.summary()

In [ ]:
# Normalize pixel values to be between 0 and 1

train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
# Compile and Train Model

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = model.fit(
    x=train_images,
    y=train_labels,
    epochs=10, 
    validation_data=(test_images, test_labels)
  )

In [ ]:
# Plot loss and accuracy

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label = 'test_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')
plt.title("Loss")
plt.show()

plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label = 'test_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.title("Accuracy")
plt.show()

# Print test accuracy
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

In [ ]:
# Visualize Prediction

def predict_single(model: tf.keras.Model, input: np.ndarray) -> int:
  """
  predict single input 
  """
  pred_logits = model.predict(np.expand_dims(input, axis=0), verbose=0)
  pred_id = np.argmax(pred_logits)
  return pred_id


# Visualize 25 test sample with image, prediction, labels
plt.figure(figsize=(10,13))
for i, (image, label) in enumerate(zip(test_images, test_labels)):
    if i == 25:
      break

    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image)

    pred_id = predict_single(model, image)
    pred_class_name = CLASS_NAMES[pred_id]
    label_class_name = CLASS_NAMES[label[0]]

    plt.xlabel(
        f"pred: {pred_class_name}\n label: {label_class_name}"
    )

In [ ]:
# Get predictions and labels all dataset

def predict_batch(model: tf.keras.Model, inputs: np.ndarray) -> np.ndarray:
  """
  predict batch input
  """
  pred_logits  = model.predict(inputs, verbose=0)
  pred_ids = np.argmax(pred_logits, axis=-1)
  return pred_ids

def get_batch_dataloader(x: np.ndarray, y: np.ndarray, batch_size: int) -> Iterator[Tuple[np.ndarray, np.ndarray]]:
  """
  Generator that yield batch of input and label.
  Each batch have N sample equal batch size
  """
  lenght = math.ceil(len(y) / batch_size)
  for i in range(lenght):
    start_index = i * batch_size
    end_index = (i + 1) * batch_size
    batch_x = x[start_index: end_index]
    batch_y = y[start_index: end_index].flatten()
    yield batch_x, batch_y


# create test_dataloder
test_dataloader = get_batch_dataloader(x=test_images, y=test_labels, batch_size=32)

# predict all data on test set
y_trues = []
y_preds = []
for i, (x, y) in enumerate(test_dataloader):
  y_trues.extend(y)
  batch_prediction = predict_batch(model=model, inputs=x)
  y_preds.extend(batch_prediction)


In [ ]:
# Display confusion matrix 

cm = confusion_matrix(y_true=y_trues, y_pred=y_preds)
disp = ConfusionMatrixDisplay(cm, display_labels=CLASS_NAMES)
fig, ax = plt.subplots(figsize=(11, 11))
disp.plot(ax=ax)

In [ ]:
# Display classification report

y_trues_name = [CLASS_NAMES[y_true] for y_true in y_trues]
y_preds_name = [CLASS_NAMES[y_pred] for y_pred in y_preds]
report = classification_report(y_true=y_trues_name, y_pred=y_preds_name, labels=CLASS_NAMES)
print(report)